<a href="https://colab.research.google.com/github/russelljjarvis/3Dprintneurons/blob/master/CollabBrian2DiffEquations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--![wsu_branding.jpg](wsu_branding.jpg)-->


<h1 align="center">
<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/wsu_branding.jpg?raw=1" width="750" height="500" />
</h1>  

<!--
# Why Brian2
* Sparse
* can be converted to fast cpp CUDA code and run on Nvidia Nano.
-->

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/russelljjarvis/LIF_DE_Brian2/blob/main/Brian2DiffEquations.ipynb)


# Assumed Knowledge (Pre-requisites)

* Phospholipid membrane bilayer
* Ionic basis of the membrane potential
* Programming with Python MATLAB or any procedural language.

# Contents 

### In this tutorial we will.

- Big picture Biology Overview of electrical excitability and trees.
    -  Electrical Signalling in Biology (trees, sea sponges, venus fly trap).
    -  Modelling issues: Reduction versus mimicry
    -  Motivation for reduced neuron models the LIF neuron.
- Solve the LIF neuron with Brian2
- Solve the LIF neuron with forward Euler and numba
- Compare models with experimental data.
- Show how the LIF model is a bad fit for biological data.
    - Better fits: Izhikevich and Adaptive Exponential Integrate and fire model.



# Other Materials:
 
* [Book (free online version) Chapter 2 only](https://neuronaldynamics.epfl.ch/online/index.html)

* [Exercises based on the book](https://neuronaldynamics-exercises.readthedocs.io/en/latest/)

* [Tutorials](https://github.com/brian-team/brian2/tree/master/tutorials)
 
* [Slides only (not the code, or code exercises](https://compneuro.neuromatch.io/tutorials/W2D3_BiologicalNeuronModels/student/W2D3_Tutorial1.html)
 





## Electrically Excitable cell membranes are common in Natural world 

* Plant cells can be electrically excitable too (for example the venus fly trap).
* Spermatoza.
* Pumpkin seeds

Reasons include fast behavior, and rapid signalling to other cells.

# Venus Fly Trap

* A plant that can trap flies by moving surprisingly fast (no skeletal muscles or motor neurons).



<h1 align="center">
<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/venus_flytrap_aps.jpg?raw=1"  width="350" height="350" />
</h1>


[Source](https://www.linkedin.com/in/ACoAAALn2tsB8Yx7K21oU-RUKtRWSjXLrXLeaDA/)

# Brain Evolution

## Synapses (chemical connnections between cells) may have evolved from very basic organisms



<h1 align="center">

<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/recropped_b_evo.png?raw=1"  width="450" height="250" />

</h1>

* Such as sea sponges

<h1 align="center">

<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/seasponge2.png?raw=1"  width="350" height="350" />
</h1>


# Trees Shapes are common in Biology.
* Nature uses trees to solve the problem of transporting information and nutrients in a volume at low metabolic cost.

* A large surface to volume ratio facilitates low cost transport.

* We don't have to model the neuronal shape (a tree) to make realistic neuronal simulations


<h1 align="center">


<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/trees5.png?raw=1" width="350" height="100"/>

</h1>




Tree roots may share nutrients and exchange information via fungal mycelium.

<h1 align="center">

<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/rhizozome2.png?raw=1"  width="450" height="250" />

<h1>

In the brain cell proteins are "walked" from rhibosomes (protein factories) along axons to their target destination at axon boutans using "axonal-transport".

Trees sample many points in a large volume, while occupying little volume themselves.

<!--![](tree3.png)
width="150" height="150" />

-->





# But we don't need the tree to do acceptably realistic simulations of neurons.

* Electrically simulating the whole tree might be bio-mimicry.

* GLIF neurons can be more accurate than Multicompartment neurons

# Biological cells Have Spatially Extended 3D form

<h1 align="center">
<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/reduction_to_lif2.png?raw=1" width="350" height="250" />
</h1>
# Many reduced neuronal models have no spatial dimensions.

* Depicted as a point, conceptually a point.

<h1 align="center">
<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/comparison_tree_LIF2.png?raw=1" width="350" height="250" />
</h1>


# Revision
## Recall ...

<h1 align="center">

<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/Slide10.jpg?raw=1" width="550" height="350" />
</h1>


# Revision
## Recall ...

<h1 align="center">

<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/Slide14.jpg?raw=1" width="550" height="350" />
</h1>

and...


<h1 align="center">

<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/water_analogy.png?raw=1" width="550" height="350" />
</h1>




# Digital Simulation as an biological lab test bench

<h1 align="center">
    <img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/virtual_experiment.png?raw=1" width="450" height="350" />
</h1>


On the time-grid of of stepsize $\Delta t$, the dynamics of Eqn. (1) is discretized to be 

\begin{eqnarray}
\tau_m \frac{\Delta V_n}{\Delta t} &=& -(V_n-V_L) +\frac{I_n}{g_L},
\end{eqnarray}

where $V_n = V(n\Delta t)$, and $I_n=I(n\Delta t)$. 

Then the updating process of $V_{n+1}$ at time $t_{n+1}=(n+1)\Delta t$ based on $V_{n}$ can be formed as 

\begin{eqnarray}
V_{n+1} &=& V_n+ \Delta V_n \\
\Delta V_n &=&  \left(-(V_n-V_L) +\frac{I}{g_L}\right)\frac{\Delta t}{\tau_m} \qquad (2)
\end{eqnarray}


## Compare Equations with Equations from Previous lecture

<h1 align="center">

<img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/digital_implementation.png?raw=1" width="325" height="125" />

</h1>

# Installs for Collab

In [ ]:
!pip install pandas -q
!pip install brian2 
!pip install plotly -q
!pip install ipywidgets -q
!git clone https://github.com/DiGyt/neuropynamics/ -q
!pip install numba
!pip uninstall --yes allensdk


fatal: destination path 'neuropynamics' already exists and is not an empty directory.
Found existing installation: allensdk 2.13.4
Uninstalling allensdk-2.13.4:
  Successfully uninstalled allensdk-2.13.4


# Interactive Code

In [ ]:
from brian2 import *
import plotly
from brian2 import NeuronGroup, SpikeMonitor, StateMonitor, run
from brian2.units import ms
import plotly.express as px
import pandas as pd
import numba
from numba import jit
import matplotlib.pyplot as plt

import numpy as np
# Brian2 package
# Unit definitions
from brian2 import mV, ms, volt, second, umetre, ufarad, siemens, cm, msiemens, amp, uA, nA
# Other stuff
from brian2 import start_scope, NeuronGroup, StateMonitor, SpikeMonitor, run
# Plotting stuff
from neuropynamics.src.utils.plotting import plot_single_neuron
# Interactive widgets


import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, HBox, VBox, Layout
# Set button layout
button_layout = Layout(width='180px', height='30px')

# Set slider config
slider_total_width = '800px'
slider_desc_width = '200px'






# Perfect Integrator


* The perfect Integrator is a stepping stone to understanding the **Leaky Integrate and Fire** model.



## Insert Perfect Integrator Equations below.

# Leaky Integrate and Fire model

# First we will simulate a LIF neuron 
the easy way using Brian2, so we know we have something to compare with


https://brian2.readthedocs.io/en/stable/introduction/brian1_to_2/library.html


# Code approach borrowed from here:

https://colab.research.google.com/github/DiGyt/neuropynamics/blob/master/notebooks/Single_neurons.ipynb#scrollTo=-BMn7Gi5EWGn

In [ ]:

def create_lif_neuron(v_max):
    """Creates a brian2 NeuronGroup that contains a single leaky integrate-and-fire neuron"""
    # Define differential equation for leaky integrate-and-fire neuron
    eqs = '''   
        dv/dt = ((El - v) + I)/tau : volt
        
        I : volt
        '''
    # Define reset function
    reset = 'v = {}*mV'.format(v_max-20)
    # Define threshold
    threshold = 'v > {}*mV'.format(v_max)
    # Return NeuronGroup object
    return NeuronGroup(1, eqs, threshold = threshold, reset = reset, method = 'euler')

def reset_lif(name):
    I_ext_slider.value = I_ext_def
    tau_slider.value = tau_def
    e_leak_slider.value = e_leak_def

# Create function that creates a neuron and plots its behavior based on the given parameters
def create_and_plot_lif_neuron(I_ext, tau, e_leak, v_max):
    # Start the scope for the brian2 toolbox to register all neurons that are created
    start_scope()
    # Define the neuron
    neuron = create_lif_neuron(v_max)      
    # Set neuron parameters
    tau = tau*ms;
    El = -e_leak*mV
    # Start monitoring the neurons state
    statemon = StateMonitor(source = neuron, variables = ['v', 'I'], record = 0)
    # Start monitoring spiking behavior
    spikemon = SpikeMonitor(source = neuron, variables= 'v')
    # Run neuron simulation for 100ms without input
    run(100*ms)
    # Set input current to neuron
    neuron.I = I_ext * mV
    # Run 500ms with input
    run(500*ms)
    # Remove input current to neuron
    neuron.I = 0 * mV
    # Run neuron simulation for 100ms without input
    run(100*ms)
    # Plot results
    plot_single_neuron(x = statemon.t/ms, neuron_data = [statemon.v[0]/mV], neuron_labels = ['Membrane Potential'], neuron_colors = ['steelblue'], \
                     spikes = spikemon.t/ms, spike_color = 'red', \
                     input_current = statemon.I[0]/mV, input_label = 'Input Current', input_color = 'gold', \
                     y_range = [-90,0], title = 'Leaky Integrate-and-Fire Neuron', x_axis_label = 'Time (ms)', y_axis_label = 'Membrane Potential', input_axis_label = 'Input Current', hline = v_max)
  


The electrical model of the neuron can fire more frequently but it can fire at different amplitudes.

# Questions:

* What will be the response of the neuron to applying more applied current?

* It is a very artificial to apply an annode to neuronal tissue.

* What natural conditions is the electrode current a substitute for in the model?

* What does changing E-leak do?

* What does changing tau do?


In [ ]:
# Set default parameters 
I_ext_def = 10.
tau_def = 20.
e_leak_def = 59.

# Create sliders for neuron parameters
tau_slider = widgets.FloatSlider(value = tau_def, min = 1., max = 30., step = 1, description = 'Charging modulation tau:', readout_format = '.1f', continuous_update = False, layout = {'width': slider_total_width}, style = {'description_width': slider_desc_width})
e_leak_slider = widgets.FloatSlider(value = e_leak_def, min = 0., max = 85., step = 1, description = 'Leakage E_leak:', readout_format = '.1f', continuous_update = False, layout = {'width': slider_total_width}, style = {'description_width': slider_desc_width}) 

# Create slider for input current
I_ext_slider = widgets.FloatSlider(value = I_ext_def, min = 0., max = 40., step = 1, description = 'Input current I:', readout_format = '.1f', continuous_update = False, layout = {'width': slider_total_width}, style = {'description_width': slider_desc_width})

# Make interactive widget for function above with the given sliders
main_widgets = interactive(create_and_plot_lif_neuron, I_ext = I_ext_slider, tau = tau_slider, e_leak = e_leak_slider, v_max = fixed(-50))


# Reset button
reset_button = widgets.Button(description='Reset', layout = button_layout)
reset_button.on_click(reset_lif)

# Display main widgets and reset button
display(VBox(children=[reset_button, main_widgets]))

<h1 align="center">
    <img src="https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/fitting_to_data.png?raw=1" width="450" height="350" />
</h1>


In [ ]:
#@jit(nopython=True) 
#@numba.jit
def fast_looping(Lt,v,V_reset,E_L,rec_spikes,tref,dt,Iinj,tau_m,g_L,tr,V_th):    
    for it in range(Lt):
      if tr > 0:  # check if in refractory period
        v[it] = V_reset  # set voltage to reset
        tr = tr - 1 # reduce running counter of refractory period
      elif v[it] >= V_th:  # if voltage over threshold
        rec_spikes.append(it)  # record spike event
        v[it] = V_reset  # reset voltage

        tr = tref / dt  # set refractory time
      # Calculate the increment of the membrane potential
      dv = (-(v[it]-E_L) + (Iinj[it]/g_L))*(dt/tau_m)
      # Update the membrane potential
      v[it + 1] = v[it] + dv

    # Get spike times in ms
    rec_spikes = np.array(rec_spikes) * dt
      
    return (v, rec_spikes)

def run_LIF(pars, Iinj, stop=False):
  """
  Simulate the LIF dynamics with external input current
  Args:
    pars       : parameter dictionary
    Iinj       : input current [pA]. The injected current here can be a value
                 or an array
    stop       : boolean. If True, use a current pulse
  Returns:
    rec_v      : membrane potential
    rec_sp     : spike times
  """

  # Set parameters
  V_th, V_reset = pars['V_th'], pars['V_reset']
  tau_m, g_L = pars['tau_m'], pars['g_L']
  V_init, E_L = pars['V_init'], pars['E_L']
  dt, range_t = pars['dt'], pars['range_t']
  Lt = range_t.size
  tref = pars['tref']

  # Initialize voltage
  v = np.zeros(Lt)
  v[0] = V_init

  # Set current time course
  Iinj = Iinj * np.ones(Lt)

  # If current pulse, set beginning and end to 0
  if stop:
    Iinj[:int(len(Iinj) / 2) - 1000] = 0
    Iinj[int(len(Iinj) / 2) + 1000:] = 0
  # Loop over time
  rec_spikes = []  # record spike times
  tr = 0.  # the count for refractory duration
  Lt = np.array(range(0,Lt - 1))
  (v, rec_spikes) = fast_looping(len(Lt),v,V_reset,E_L,rec_spikes,tref,dt,Iinj,tau_m,g_L,tr,V_th)
  pars['Iinj'] =  Iinj

  return (v, rec_spikes,pars)

def default_pars(**kwargs):
  pars = {}

  # typical neuron parameters#
  pars['V_th'] = -55.     # spike threshold [mV]
  pars['V_reset'] = -75.  # reset potential [mV]
  pars['tau_m'] = 10.     # membrane time constant [ms]
  pars['g_L'] = 10.       # leak conductance [nS]
  pars['V_init'] = -75.   # initial potential [mV]
  pars['E_L'] = -75.      # leak reversal potential [mV]
  pars['tref'] = 2.       # refractory time (ms)

  # simulation parameters #
  pars['T'] = 440.  # Total duration of simulation [ms]
  pars['dt'] = .1   # Simulation time step [ms]

  # external parameters if any #
  for k in kwargs:
    pars[k] = kwargs[k]

  pars['range_t'] = np.arange(0, pars['T'], pars['dt'])  # Vector of discretized time points [ms]

  return pars


pars = default_pars()

# Simulate LIF model
v, sp, pars = run_LIF(pars, Iinj=210, stop=True)

# Visualize

def plot_volt_trace(pars, v, sp):
    """
    Plot trajetory of membrane potential for a single neuron

    Expects:
    pars   : parameter dictionary
    v      : volt trajetory
    sp     : spike train

    Returns:
    figure of the membrane potential trajetory for a single neuron
    """

    V_th = pars['V_th']
    dt, range_t = pars['dt'], pars['range_t']
    if sp.size:
      sp_num = (sp / dt).astype(int) - 1
      v[sp_num] += 20  # draw nicer spikes


    fig = plotly.express.line(x=pars['range_t'],y=v)
    fig.show()

    #fig = plotly.express.line(x=pars['range_t'],y=pars['Iinj'])
    #fig.show()

    #plt.plot(pars['range_t'], v, 'b')
  
    #plt.axhline(V_th, 0, 1, color='k', ls='--')
    
    
    #plt.plot(pars['range_t'],pars['Iinj'], 'g')
    #plt.show()
    #plt.xlabel('Time (ms)')
    #plt.ylabel('V (mV)')
    #plt.legend(['Membrane\npotential', r'Threshold V$_{\mathrm{th}}$'],
     #        loc=[1.05, 0.75])


    #if np.min(v)>-90:
    #  plt.ylim([-90, -0])
    #else:
     # plt.ylim([np.min(v)-10, -0])








## This is what is known as a "regular spiking model"

In [ ]:
plot_volt_trace(pars, v, sp)


In [ ]:

# Set default parameters 
I_ext_def = 202.
tau_def = 10.
e_leak_def = -75.


@widgets.interact(
    I_dc = widgets.FloatSlider(value = I_ext_def, min = 0., max = 300., step = 1,
                               description = 'Input Current Injection (Amps):',
                               continuous_update = False, 
                               layout = {'width': slider_total_width}, 
                               style = {'description_width': slider_desc_width}),
    
    tau_m = widgets.FloatSlider(value = tau_def, min = 1., max = 20., step = 1, 
                                description = 'Charging modulation tau:', readout_format = '.1f', 
                                continuous_update = False, 
                                layout = {'width': slider_total_width}, style = {'description_width': slider_desc_width}),
    
    E_L = widgets.FloatSlider(value = e_leak_def, min = -85., max = 0., step = 1, 
                                description = 'Leakage E_leak:', readout_format = '.1f', 
                                continuous_update = False, 
                                layout = {'width': slider_total_width}, style = {'description_width': slider_desc_width}), 
    )



def diff_DC(I_dc=I_ext_def, tau_m=tau_def,E_L=e_leak_def):
    pars = default_pars(T=1000.)
    pars['tau_m'] = tau_m
    pars['E_L'] = E_L
    #print(I_dc)
    v, sp, pars = run_LIF(pars, Iinj=I_dc)
    plot_volt_trace(pars, v, sp)
    plt.show()



interactive(children=(FloatSlider(value=202.0, continuous_update=False, description='Input Current Injection (…

## What is wrong

* The plot compares the trace of a Leaky Integrate and Fire Neuron against a more bio-realistic Hodgkin Huxley model. 

* What metrics could you use to quantify the agreement/dis-agreement between the two?

* Do you think the agreement is acceptable?

* What is the least realistic part of the LIF $V_{m}$ shape?

* What should be the real units of $V_{m}$?

* What should be the the real resting membrane potential $V_{R}$?


![](https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/LIF_vs_HH.png?raw=1)

### Fitting to data.

* the Action Potential shape may still disagree with data in all of these ways:

![](https://github.com/russelljjarvis/LIF_DE_Brian2/blob/main/other_lecture/what_is_wrong.png?raw=1)